# Welcome to a qMRLab interactive blog post Jupyter Notebook!

If this is your first time running a Juptyer Notebook, there's a lot of tutorials available online to help. [Here's one](https://www.dataquest.io/blog/jupyter-notebook-tutorial/) for your convenience.

## Introduction

This notebook contains everything needed to reproduce the Actual Flip-Angle B<sub>1</sub> blog post on the [qMRLab website](). In fact, this notebook generated the HTML for the blog post too! This notebook is currently running on a MyBinder server that only you can access, but if you want to be kept up-to-date on any changes that the developpers make to this notebook, you should go to it's [GitHub repository](https://github.com/qMRLab/t1_notebooks) and follow it by clicking the "Watch" button in the top right (you may need to create a GitHub account, if you don't have one already).

## Tips

Here's a few things you can do in this notebook

### Code
* Run the entire processing by clicking above on the "Kernel" tab, then "Restart & Run All". It will be complete when none of the cells have an asterix "\*" in the square brackets.
* To change the code, you need to click once on code cells. To re-run that cell, click the "Run" button above when the cell is selected.
  * **Note:** Cells can depend on previous cells, or even on previous runs of the cell itself, so it's best to run all the previous cells beforehand.
* This binder runs on SoS, which allows the mixing of Octave (i.e. an open-source MATLAB) and Python cells. Take a look a the drop down menu on the top right of the cells to know which one you are running.
* To transfer data from cells of one language to another, you need to create a new cell in the incoming language and run `%get (param name) --from (outgoing language)`. See cells below for several examples within this notebook.

### HTML
* To reproduce the HTML of the blog post, run the entire processing pipeline (see point one in the previous section), then save the notebook (save icon, top left). Now, click on the drop down menu on the left pannel, and select `%sossave --to html --force` . After a few seconds, it should output "Workflow saved to ActualFlipAngleImaging.html" – click on the HTML name, and you're done!
* Cells with tags called "scratch" are not displayed in the generated HTML.
* Cells with the tag "report_output" display the output (e.g. figures) in the generated HTML.
* Currently in an un-run notebook, the HTML is not formatted like the website. To do so, run the Python module import cell (`# Module imports`) and then very last cell (`display(HTML(...)`).

**If you have any other questions or comments, please raise them in a [GitHub issue](https://github.com/qMRLab/t1_notebooks/issues).**

# Note

The following cell is meant to be displayed for instructional purposes in the blog post HTML when "All cells" gets displayed (i.e. the Octave code).

In [ ]:
% **Blog post code introduction**
% 
% Congrats on activating the "All cells" option in this interactive blog post =D
%
% Below, several new HTML blocks have appears prior to the figures, displaying the Octave/MATLAB code that was used to generate the figures in this blog post.
%
% If you want to reproduce the data on your own local computer, you simply need to have qMRLab installed in your Octave/MATLAB path and run the "startup.m" file, as is shown below.
%
% If you want to get under the hood and modify the code right now, you can do so in the Jupyter Notebook of this blog post hosted on MyBinder. The link to it is in the introduction above.

In [ ]:
# PYTHON CODE
# Module imports

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

from IPython.core.display import display, HTML

<center><h1 style="font-family: timesnewroman;font-size: 40px;">MTR and MTsat</h1></center>
<p>

<div class=blog_body>
<p style="text-align:justify;">
Transmit radiofrequency transmit field maps (B<sub>1</sub><sup>+</sup>, or B<sub>1</sub> for short) are used in diverse applications in MRI including: the study of electrical properties in tissues in vivo (Sled & Pike 1998; Katscher et al. 2009), specific absorption rate (SAR) calculations (Ibrahim et al. 2001), the calibration of quantitative T<sub>1</sub> (Deoni 2007; Boudreau et al. 2017) and T<sub>2</sub> (Sled and Pike 2000) maps, better parameter estimation from magnetization transfer measurements (Ropele et al. 2005; Boudreau et al. 2018), B<sub>1</sub> shimming to improve image quality at whole-body ultra high fields (van den Bergen et al. 2007), or quality control of RF coils (Yarnykh 2007).
</p>
</div>

<center> <h2 style="font-family:timesnewroman;font-size:30px">Signal Modelling</h2> </center>

<div class=blog_body>
<p style="text-align:justify;">
The pulse sequence of the AFI method (Figure 1) is composed with two identical RF pulses and two different delays (TR<sub>1</sub> < TR<sub>2</sub>). After each RF pulse, the signal intensity is acquired followed by a spoiler to destroy the residual transverse magnetization prior to the following RF pulse. This method implements a pulsed steady-state signal with a gradient-echo acquisition, thus preventing the use of long repetition times (Yarnykh 2007). It has been demonstrated that if delays TR<sub>1</sub> and TR<sub>2</sub> are sufficiently short (e.g. TR<sub>1</sub>/TR<sub>2</sub> = 20 ms/100 ms), and the transverse magnetization is completely spoiled, the ratio of signal intensities (r = S<sub>2</sub>/S<sub>1</sub>) depends on the flip angle of applied pulses and is highly insensitive to T<sub>1</sub> (Yarnykh 2007). 
</p>
</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 1. Simplified pulse sequence diagram of an actual flip-angle imaging (AFI) pulse sequence with a gradient echo readout. TR<sub>1</sub>: repetition time 1, TR<sub>2</sub>: repetition time 2, <i>θ</i>: excitation flip angle for the measurement, IMG: image acquisition (k-space readout), SPOIL: spoiler gradient.
</b>
</p>
</div>

<p>
<center><img src="afi_pulsesequence.png" style="width:500px;height:auto;"></center>

<div class=blog_body>
<p style="text-align:justify;">
The magnetization of an AFI experiment can be modeled under steady-state conditions by the implementation of a fast repetition of the sequence (TR<sub>1</sub> < TR<sub>2</sub> < T<sub>1</sub>). The solution of the Bloch equations for the AFI method is given by Equations 1 and 2 that represent the longitudinal magnetization before the application of the RF pulses:
</p>

<p style="text-align:justify;">
<center><img src="equation1.png" style="width:800px;height:80px;margin-bottom: 50px;margin-top: 50px;"></center>
</p>
    
<p style="text-align:justify;">
<center><img src="equation2.png" style="width:800px;height:85px;margin-bottom: 50px;margin-top: 50px;"></center>
</p>

<p style="text-align:justify;">
<i>M<sub>z1,2</sub></i> is the longitudinal magnetization of both pulses, <i>M</i><sub>0</sub> is the magnetization at thermal equilibrium, TR<sub>1</sub> is the delay time after the first pulse, TR<sub>2</sub> is the delay time after the second identical pulse (Figure 1), and <i>θ</i> is the excitation flip angle. The steady state longitudinal magnetization <i>M<sub>z</sub></i> curves of different T<sub>1</sub> values for a range of <i>θ<sub>n</sub></i> and TRs values are shown in Figure 2.
</p>

</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 2. Longitudinal magnetization before the first radiofrequency pulse (Equation 1, solid lines) and before the second identical pulse (Equation 2, dashed lines) for three different T<sub>1</sub> values.
</b>
</p>
</div>

In [ ]:
%% MATLAB/OCTAVE CODE
% Adds qMRLab to the path of the environment

cd ../qMRLab
startup

In [ ]:
%% Get MTR and MTsat varying MT pulse shape
% Simulate complete qMT-SPGR protocol
Model_qmtSPGR = qmt_spgr;
Model_qmtSPGR.Prot.MTdata.Mat = [490, 1200]; % 490�, 1.2 kHz

% Timing - TR = 32 ms
Model_qmtSPGR.Prot.TimingTable.Mat(1) = 0.0120;
Model_qmtSPGR.Prot.TimingTable.Mat(2) = 0.0032;
Model_qmtSPGR.Prot.TimingTable.Mat(3) = 0.0018;
Model_qmtSPGR.Prot.TimingTable.Mat(4) = 0.0150;
Model_qmtSPGR.Prot.TimingTable.Mat(5) = 0.0320;

Model_qmtSPGR.options.Readpulsealpha = 6;

% Set simulation options
Opt.Method = 'Analytical equation';
Opt.ResetMz = false;
Opt.SNR = 1000;

% Input parameters
% MTon
x = struct;
x.F = 0.16;
x.kr = 30;
x.R1f = 1;
x.R1r = 1;
x.T2f = 0.03;
x.T2r = 1.3e-05;

% Timing parameters for vfa_t1.analytical_solution function
% PDw - MToff
PDw_Model = vfa_t1;
paramsPDw.EXC_FA = 6;
paramsPDw.TR = 32; % ms

% T1w
T1w_Model = vfa_t1; 
paramsT1w.EXC_FA = 20;
paramsT1w.TR = 18; % ms

% Timing parameters for MTSAT_exec function
PDParams(1) = paramsPDw.EXC_FA;
PDParams(2) = paramsPDw.TR;
T1Params(1) = paramsT1w.EXC_FA;
T1Params(2) = paramsT1w.TR;
MTParams(1) = 6;
MTParams(2) = 32;
B1Params(1) = 1;

% Four F values (WM, NAWM, LESION I, II
F = [0.20, 0.15, 0.10, 0.09];

%%%%%%% Effect of the MT pulse shape %%%%%%%
% MT pulse options
MTpulses = {'gaussian', 'gausshan', 'sinc','sincgauss','fermi'};

% Get MTR and MTsat using different MT pulses
for ii=1:length(MTpulses)
    for jj=1:length(F)
        % Hard pulse
        if strcmp(MTpulses{ii},'hard')
            Model_qmtSPGR.options.MT_Pulse_Shape = MTpulses{ii};
        end
        
        % Gaussian pulse and Gaussian pulse with Hanning window
        if (strcmp(MTpulses{ii},'gaussian') || strcmp(MTpulses{ii},'gausshann'))
            Model_qmtSPGR.options.MT_Pulse_Bandwidth = 200;
            Model_qmtSPGR.options.MT_Pulse_Shape = MTpulses{ii};
        end
        
        % Sinc pulse and Sinc pulse with Hanning window
        if (strcmp(MTpulses{ii},'sinc') || strcmp(MTpulses{ii},'sinchann'))
            Model_qmtSPGR.options.MT_Pulse_SincTBW = 4;
            Model_qmtSPGR.options.MT_Pulse_Shape = MTpulses{ii};
        end
        
        % Sincgauss pulse
        if strcmp(MTpulses{ii},'sincgauss')
            Model_qmtSPGR.options.MT_Pulse_SincTBW = 4;
            Model_qmtSPGR.options.MT_Pulse_Bandwidth = 200;
            Model_qmtSPGR.options.MT_Pulse_Shape = MTpulses{ii};
        end
        
        % Fermi pulse
        if strcmp(MTpulses{ii},'fermi')
            % MT pulse fermi transition Tmt/33.81
            Model_qmtSPGR.options.MT_Pulse_Fermitransitiona = Model_qmtSPGR.Prot.TimingTable.Mat(1)/33.81;
            %Model_qmtSPGR.options.MT_Pulse_Fermitransitiona = 0.35;
            Model_qmtSPGR.options.MT_Pulse_Shape = MTpulses{ii};
        end
        
        % Signal
        x.F = F(jj);
        Signal_qmtSPGR = equation(Model_qmtSPGR, x, Opt);
        % PDw - MToff
        paramsPDw.T1 = 1/x.R1f*1000; % ms
        PDw = vfa_t1.analytical_solution(paramsPDw);
        % T1w
        paramsT1w.T1 = 1/x.R1f*1000; % ms
        T1w = vfa_t1.analytical_solution(paramsT1w);
        % MTon
        MT = Signal_qmtSPGR*PDw;

        % MTR calculation
        MTR_MT(ii,jj) = 100*(PDw - MT)/PDw;
        
        % MTsat calculation
        dataMTsat.PDw = PDw;
        dataMTsat.T1w = T1w;
        dataMTsat.MTw = MT;
        [MTsaturation,~] = MTSAT_exec(dataMTsat, MTParams, PDParams, T1Params, B1Params);
        MTsat_MT(ii,jj) = MTsaturation;
    end
end

In [ ]:
%get F --from Octave
%get MTpulses --from Octave
%get MTR_MT --from Octave
%get MTsat_MT --from Octave

In [ ]:
# PYTHON CODE
init_notebook_mode(connected=True)

data_MTR = [dict(
    visible = False,
    line=dict(color='royalblue'),
    x = F,
    y = abs(np.squeeze(np.asarray(MTR_MT[ii,:]))),
    name = 'MTR',
    text = 'MTR (%)',
    hoverinfo = 'x+y+text') for ii in range(len(MTpulses))]

data_MTR[0]['visible'] = True

data_MTsat = [dict(
    visible = False,
    line=dict(color='firebrick'),
    x = F,
    y = abs(np.squeeze(np.asarray(MTsat_MT[ii,:]))),
    name = 'MTsat',
    text = 'MTsat (a.u.)',
    xaxis = 'x',
    yaxis = 'y2',
    hoverinfo = 'x+y+text') for ii in range(len(MTpulses))]

data_MTsat[0]['visible'] = True

data = data_MTR + data_MTsat

buttons = []
for i, label in enumerate(MTpulses):
    visibility = [False] * len(MTpulses)*len(data)
    for j in np.arange(0,len(MTpulses)*len(data),len(MTpulses)):
        visibility[i+j] = True
    button = dict(
                label =  label,
                method = 'update',
                args = [{'visible': visibility}])
    buttons.append(button)
        
updatemenus = list([
    dict(active=0,
         x=0.3,
         y=1.1,
         buttons=buttons
    )
])

layout = go.Layout(
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=80,
        b=60,
        t=60,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='F',
            font=dict(
                family='Times New Roman',
                size=22),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='MTR (%)',
            font=dict(
                family='Times New Roman',
                size=22,
                color='royalblue'),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.5004254919715793,
            y=1.15,
            showarrow=False,
            text=False,
            visible=False,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.14,
            y=0.5,
            showarrow=False,
            text='MTsat (a.u.)',
            font=dict(
                family='Times New Roman',
                size=22,
                color=('firebrick')),
            textangle=-90,
            xref='paper',
            yref='paper'
        )
    ],
    xaxis=dict(
        autorange=False,
        range=[0, F[0]+0.1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=True,
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    xaxis2=dict(
        autorange=False,
        range=[0, F[0]+0.1],
        showgrid=False,
        showticklabels=False,
        mirror=True,
        overlaying= 'x',
        anchor= 'y2',
        side= 'top',
        linecolor='black',
        linewidth=2
    ),
    yaxis2=dict(
        autorange=True,
        showgrid=False,
        overlaying= 'y',
        anchor= 'x',
        side= 'right',
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.7,
        y=0.5,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)
layout['updatemenus'] = updatemenus

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

In [ ]:
%% Get MTR and MTsat varying off-resonance frequency
% Simulate complete qMT-SPGR protocol
Model_qmtSPGR = qmt_spgr;
Model_qmtSPGR.Prot.MTdata.Mat = [490, 1200]; % 490�, 1.2 kHz

% Timing - TR = 32 ms
Model_qmtSPGR.Prot.TimingTable.Mat(1) = 0.0120;
Model_qmtSPGR.Prot.TimingTable.Mat(2) = 0.0032;
Model_qmtSPGR.Prot.TimingTable.Mat(3) = 0.0018;
Model_qmtSPGR.Prot.TimingTable.Mat(4) = 0.0150;
Model_qmtSPGR.Prot.TimingTable.Mat(5) = 0.0320;

Model_qmtSPGR.options.Readpulsealpha = 6;

% Set simulation options
Opt.Method = 'Analytical equation';
Opt.ResetMz = false;
Opt.SNR = 1000;

% Input parameters
% MTon
x = struct;
x.F = 0.16;
x.kr = 30;
x.R1f = 1;
x.R1r = 1;
x.T2f = 0.03;
x.T2r = 1.3e-05;

% Timing parameters for vfa_t1.analytical_solution function
% PDw - MToff
PDw_Model = vfa_t1;
paramsPDw.EXC_FA = 6;
paramsPDw.TR = 32; % ms

% T1w
T1w_Model = vfa_t1; 
paramsT1w.EXC_FA = 20;
paramsT1w.TR = 18; % ms

% Timing parameters for MTSAT_exec function
PDParams(1) = paramsPDw.EXC_FA;
PDParams(2) = paramsPDw.TR;
T1Params(1) = paramsT1w.EXC_FA;
T1Params(2) = paramsT1w.TR;
MTParams(1) = 6;
MTParams(2) = 32;
B1Params(1) = 1;

% Four F values (WM, NAWM, LESION I, II
F = [0.20, 0.15, 0.10, 0.09];

%%%%%%% Effect of the off-resonance pulse %%%%%%%
% Off-resonance values
offResonance = 600:600:6600;

% Get MTR and MTsat using different off-resonance frequency
for ii=1:length(offResonance)
    for jj=1:length(F)
        % Set params
        x.F = F(jj);
        Model_qmtSPGR.Prot.MTdata.Mat = [490, offResonance(ii)];
        
        % Signal
        Signal_qmtSPGR = equation(Model_qmtSPGR, x, Opt);
        % PDw - MToff
        paramsPDw.T1 = 1/x.R1f*1000; % ms
        PDw = vfa_t1.analytical_solution(paramsPDw);
        % T1w
        paramsT1w.T1 = 1/x.R1f*1000; % ms
        T1w = vfa_t1.analytical_solution(paramsT1w);
        % MTon
        MT = Signal_qmtSPGR*PDw;

        % MTR calculation
        MTR_offRes(ii,jj) = 100*(PDw - MT)/PDw;
        
        % MTsat calculation
        dataMTsat.PDw = PDw;
        dataMTsat.T1w = T1w;
        dataMTsat.MTw = MT;
        [MTsaturation,~] = MTSAT_exec(dataMTsat, MTParams, PDParams, T1Params, B1Params);
        MTsat_offRes(ii,jj) = MTsaturation;
    end
end

In [ ]:
%get F --from Octave
%get offResonance --from Octave
%get MTR_offRes --from Octave
%get MTsat_offRes --from Octave

In [ ]:
# PYTHON CODE
init_notebook_mode(connected=True)

data_MTR = [dict(
    visible = False,
    line=dict(color='royalblue'),
    x = F,
    y = abs(np.squeeze(np.asarray(MTR_offRes[ii,:]))),
    name = 'MTR',
    text = 'MTR (%)',
    hoverinfo = 'x+y+text') for ii in range(len(offResonance))]

data_MTR[0]['visible'] = True

data_MTsat = [dict(
    visible = False,
    line=dict(color='firebrick'),
    x = F,
    y = abs(np.squeeze(np.asarray(MTsat_offRes[ii,:]))),
    name = 'MTsat',
    text = 'MTsat (a.u.)',
    xaxis = 'x',
    yaxis = 'y2',
    hoverinfo = 'x+y+text') for ii in range(len(offResonance))]

data_MTsat[0]['visible'] = True

data = data_MTR + data_MTsat

steps = []
for i in range(len(offResonance)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(offResonance)],
        label = str(offResonance[i])
    )
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.0,
    active = 0,
    currentvalue = {"prefix": "Offset frequency: <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps
)]

layout = go.Layout(
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=80,
        b=60,
        t=60,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='F',
            font=dict(
                family='Times New Roman',
                size=22),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='MTR (%)',
            font=dict(
                family='Times New Roman',
                size=22,
                color='royalblue'),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.5004254919715793,
            y=1.15,
            showarrow=False,
            text=False,
            visible=False,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.14,
            y=0.5,
            showarrow=False,
            text='MTsat (a.u.)',
            font=dict(
                family='Times New Roman',
                size=22,
                color=('firebrick')),
            textangle=-90,
            xref='paper',
            yref='paper'
        )
    ],
    xaxis=dict(
        autorange=False,
        range=[0, F[0]+0.1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=True,
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    xaxis2=dict(
        autorange=False,
        range=[0, F[0]+0.1],
        showgrid=False,
        showticklabels=False,
        mirror=True,
        overlaying= 'x',
        anchor= 'y2',
        side= 'top',
        linecolor='black',
        linewidth=2
    ),
    yaxis2=dict(
        autorange=True,
        showgrid=False,
        overlaying= 'y',
        anchor= 'x',
        side= 'right',
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.7,
        y=0.5,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ),
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

In [ ]:
%% Get MTR and MTsat varying the TR
% Simulate complete qMT-SPGR protocol
Model_qmtSPGR = qmt_spgr;
Model_qmtSPGR.Prot.MTdata.Mat = [490, 1200]; % 490�, 1.2 kHz

% Timing - TR = 32 ms
Model_qmtSPGR.Prot.TimingTable.Mat(1) = 0.0120;
Model_qmtSPGR.Prot.TimingTable.Mat(2) = 0.0032;
Model_qmtSPGR.Prot.TimingTable.Mat(3) = 0.0018;
Model_qmtSPGR.Prot.TimingTable.Mat(4) = 0.0150;
Model_qmtSPGR.Prot.TimingTable.Mat(5) = 0.0320;

Model_qmtSPGR.options.Readpulsealpha = 6;

% Set simulation options
Opt.Method = 'Analytical equation';
Opt.ResetMz = false;
Opt.SNR = 1000;

% Input parameters
% MTon
x = struct;
x.F = 0.16;
x.kr = 30;
x.R1f = 1;
x.R1r = 1;
x.T2f = 0.03;
x.T2r = 1.3e-05;

% Timing parameters for vfa_t1.analytical_solution function
% PDw - MToff
PDw_Model = vfa_t1;
paramsPDw.EXC_FA = 6;
paramsPDw.TR = 32; % ms

% T1w
T1w_Model = vfa_t1; 
paramsT1w.EXC_FA = 20;
paramsT1w.TR = 18; % ms

% Timing parameters for MTSAT_exec function
PDParams(1) = paramsPDw.EXC_FA;
PDParams(2) = paramsPDw.TR;
T1Params(1) = paramsT1w.EXC_FA;
T1Params(2) = paramsT1w.TR;
MTParams(1) = 6;
MTParams(2) = 32;
B1Params(1) = 1;

% Four F values (WM, NAWM, LESION I, II
F = [0.20, 0.15, 0.10, 0.09];

%%%%%%% Effect of the repetition time TR %%%%%%%
% TRs
TRs = 24:2:40;

% Get MTR and MTsat using different MT pulses
for ii=1:length(TRs)
    for jj=1:length(F)
        % Set params
        x.F = F(jj);
        Model_qmtSPGR.Prot.TimingTable.Mat(5) = TRs(ii)/1000;
        Model_qmtSPGR.Prot.TimingTable.Mat(1) = Model_qmtSPGR.Prot.TimingTable.Mat(5) - Model_qmtSPGR.Prot.TimingTable.Mat(2) - Model_qmtSPGR.Prot.TimingTable.Mat(3) - Model_qmtSPGR.Prot.TimingTable.Mat(4);
        paramsPDw.TR = TRs(ii);
        MTparams(2) = TRs(ii);

        % Signal
        Signal_qmtSPGR = equation(Model_qmtSPGR, x, Opt);
        % PDw - MToff
        paramsPDw.T1 = 1/x.R1f*1000; % ms
        PDw = vfa_t1.analytical_solution(paramsPDw);
        % T1w
        paramsT1w.T1 = 1/x.R1f*1000; % ms
        T1w = vfa_t1.analytical_solution(paramsT1w);
        % MTon
        MT = Signal_qmtSPGR*PDw;

        % MTR calculation
        MTR_TR(ii,jj) = 100*(PDw - MT)/PDw;
        
        % MTsat calculation
        dataMTsat.PDw = PDw;
        dataMTsat.T1w = T1w;
        dataMTsat.MTw = MT;
        [MTsaturation,~] = MTSAT_exec(dataMTsat, MTParams, PDParams, T1Params, B1Params);
        MTsat_TR(ii,jj) = MTsaturation;
    end
end

In [ ]:
%get F --from Octave
%get TRs --from Octave
%get MTR_TR --from Octave
%get MTsat_TR --from Octave

In [ ]:
# PYTHON CODE
init_notebook_mode(connected=True)

data_MTR = [dict(
    visible = False,
    line=dict(color='royalblue'),
    x = F,
    y = abs(np.squeeze(np.asarray(MTR_TR[ii,:]))),
    name = 'MTR',
    text = 'MTR (%)',
    hoverinfo = 'x+y+text') for ii in range(len(TRs))]

data_MTR[0]['visible'] = True

data_MTsat = [dict(
    visible = False,
    line=dict(color='firebrick'),
    x = F,
    y = abs(np.squeeze(np.asarray(MTsat_TR[ii,:]))),
    name = 'MTsat',
    text = 'MTsat (a.u.)',
    xaxis = 'x',
    yaxis = 'y2',
    hoverinfo = 'x+y+text') for ii in range(len(TRs))]

data_MTsat[0]['visible'] = True

data = data_MTR + data_MTsat

steps = []
for i in range(len(TRs)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(TRs)],
        label = str(TRs[i])
    )
    step['args'][1][i] = True
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.0,
    active = 0,
    currentvalue = {"prefix": "Repetition time: <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps
)]

layout = go.Layout(
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=80,
        b=60,
        t=60,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='F',
            font=dict(
                family='Times New Roman',
                size=22),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='MTR (%)',
            font=dict(
                family='Times New Roman',
                size=22,
                color='royalblue'),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.5004254919715793,
            y=1.15,
            showarrow=False,
            text=False,
            visible=False,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.14,
            y=0.5,
            showarrow=False,
            text='MTsat (a.u.)',
            font=dict(
                family='Times New Roman',
                size=22,
                color=('firebrick')),
            textangle=-90,
            xref='paper',
            yref='paper'
        )
    ],
    xaxis=dict(
        autorange=False,
        range=[0, F[0]+0.1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0,100],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    xaxis2=dict(
        autorange=False,
        range=[0, F[0]+0.1],
        showgrid=False,
        showticklabels=False,
        mirror=True,
        overlaying= 'x',
        anchor= 'y2',
        side= 'top',
        linecolor='black',
        linewidth=2
    ),
    yaxis2=dict(
        autorange=False,
        range=[0, 20],
        showgrid=False,
        overlaying= 'y',
        anchor= 'x',
        side= 'right',
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.7,
        y=0.5,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ),
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

<div class=blog_body>

<p style="text-align:justify;">
The analytical solution of the Bloch equations in a steady state experiment (Equation 1 and Equation 2) makes several assumptions leading to practical challenges. First, it is assumed that the longitudinal magnetization has reached a steady state after a large number of repetition times (TR), and that the transverse magnetization is perfectly spoiled prior to each pulse. To explore these properties, a numerical approach known as Bloch simulations are used to estimate the signal from an MRI experiment given a set of sequence parameters. Here, the Bloch simulations allow us to estimate the magnetization using a different number of sequence repetition, and look at a special case when the steady-state is not achieved (small number of sequence repetition). As can be seen in Figure 3, the number of repetitions required to reach a steady state not only depends on T<sub>1</sub>, but also on the flip angle.
</p>
</div>

In [ ]:
%% Get MTR and MTsat varying F and T1
% Simulate complete qMT-SPGR protocol
Model_qmtSPGR = qmt_spgr;
Model_qmtSPGR.Prot.MTdata.Mat = [490, 1200]; % 490�, 1.2 kHz

% Timing - TR = 32 ms
Model_qmtSPGR.Prot.TimingTable.Mat(1) = 0.0120;
Model_qmtSPGR.Prot.TimingTable.Mat(2) = 0.0032;
Model_qmtSPGR.Prot.TimingTable.Mat(3) = 0.0018;
Model_qmtSPGR.Prot.TimingTable.Mat(4) = 0.0150;
Model_qmtSPGR.Prot.TimingTable.Mat(5) = 0.0320;

Model_qmtSPGR.options.Readpulsealpha = 6;

% Set simulation options
Opt.Method = 'Analytical equation';
Opt.ResetMz = false;
Opt.SNR = 1000;

% Input parameters
% MTon
x = struct;
x.F = 0.16; % Vary F
x.kr = 30;
x.R1f = 1; % Vary R1f
x.R1r = 1;
x.T2f = 0.03;
x.T2r = 1.3e-05;

% Timing parameters for vfa_t1.analytical_solution function
% PDw - MToff
PDw_Model = vfa_t1;
paramsPDw.EXC_FA = 6;
paramsPDw.TR = 32; % ms

% T1w
T1w_Model = vfa_t1; 
paramsT1w.EXC_FA = 20;
paramsT1w.TR = 18; % ms

% Timing parameters for MTSAT_exec function
PDParams(1) = paramsPDw.EXC_FA;
PDParams(2) = paramsPDw.TR;
T1Params(1) = paramsT1w.EXC_FA;
T1Params(2) = paramsT1w.TR;
MTParams(1) = 6;
MTParams(2) = 32;
B1Params(1) = 1;

%%%%%%% EFFECTS OF F AND R1F %%%%%%%
F = 0.05:0.05:0.40;
T1 = 0.5:0.1:1.9;
R1f = 1./T1;

% Define zero-percMatrices
percMTR = zeros(length(F),length(R1f));
percMTsat = zeros(length(F),length(R1f));

% Get signal using different F and T1w
for ii=1:length(F)
    for jj=1:length(R1f)
        % Update input params
        x.F = F(ii);
        x.R1f = R1f(jj);
        % Signal
        Signal_qmtSPGR = equation(Model_qmtSPGR, x, Opt);
        % PDw - MToff
        paramsPDw.T1 = 1/x.R1f*1000; % ms
        PDw = vfa_t1.analytical_solution(paramsPDw);
        % T1w
        paramsT1w.T1 = 1/x.R1f*1000; % ms
        T1w = vfa_t1.analytical_solution(paramsT1w);
        % MTon
        MT = Signal_qmtSPGR*PDw;
        
        % MTR calculation
        MTR(ii,jj) = 100*(PDw - MT)/PDw;
        
        % MTsat calculation
        dataMTsat.PDw = PDw;
        dataMTsat.T1w = T1w;
        dataMTsat.MTw = MT;
        [MTsaturation,~] = MTSAT_exec(dataMTsat, MTParams, PDParams, T1Params, B1Params);
        MTsat(ii,jj) = MTsaturation;
        
        if jj>1
            % Calculate percentage differences
            percMTR(ii,jj) = abs(MTR(ii,jj) - MTR(ii,jj-1))*100/MTR(ii,jj);
            percMTsat(ii,jj) = abs(MTsat(ii,jj) - MTsat(ii,jj-1))*100/MTsat(ii,jj);
        end
    end
end

In [ ]:
%get F --from Octave
%get T1 --from Octave
%get percMTR --from Octave
%get percMTsat --from Octave

In [ ]:
# PYTHON CODE
init_notebook_mode(connected=True)

data_MTR = [dict(
    visible = False,
    line=dict(color='royalblue'),
    x = T1,
    y = abs(np.squeeze(np.asarray(percMTR[ii,:]))),
    name = 'MTR',
    text = 'MTR (%)',
    hoverinfo = 'x+y+text') for ii in range(len(F))]

data_MTR[0]['visible'] = True

data_MTsat = [dict(
    visible = False,
    line=dict(color='firebrick'),
    x = T1,
    y = abs(np.squeeze(np.asarray(percMTsat[ii,:]))),
    name = 'MTsat',
    text = 'MTsat (a.u.)',
    xaxis='x2',
    yaxis='y2',
    hoverinfo = 'x+y+text') for ii in range(len(F))]

data_MTsat[0]['visible'] = True

data = data_MTR + data_MTsat

steps = []
for i in range(len(F)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(F)],
        label = str(F[i])
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.0,
    active = 0,
    currentvalue = {"prefix": "F: <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps
)]

layout = go.Layout(
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=80,
        b=60,
        t=60,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='T<sub>1</sub> (s)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='MTR (%)',
            font=dict(
                family='Times New Roman',
                size=22,
                color='royalblue'
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.5004254919715793,
            y=1.15,
            showarrow=False,
            text=False,
            visible=False,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.16,
            y=0.5,
            showarrow=False,
            text='MTsat (a.u.)',
            font=dict(
                family='Times New Roman',
                size=22,
                color='firebrick'
            ),
            xref='paper',
            yref='paper',
            textangle=-90,
        )
    ],
    xaxis=dict(
        autorange=False,
        range=[0, T1[-1]+0.2],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0,15],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    xaxis2=dict(
        autorange=False,
        range=[0, T1[-1]+0.2],
        showgrid=False,
        showticklabels=False,
        mirror=True,
        overlaying= 'x',
        anchor= 'y2',
        side= 'top',
        linecolor='black',
        linewidth=2
    ),
    yaxis2=dict(
        autorange=False,
        range=[0, 15],
        showgrid=False,
        overlaying= 'y',
        anchor= 'x',
        side= 'right',
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.05,
        y=0.9,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ),
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 3. Signal 1 (blue) and Signal 2 (red) curves simulated using Bloch simulations (solid lines) for a number of repetitions ranging from 1 to 150, plotted against the ideal case (Equations 1 and 2 – dashed lines). Simulation details:  TR<sub>1</sub> = 20 ms, TR<sub>2</sub> = 100 ms, T<sub>1</sub> = 900 ms, 100 spins. Ideal spoiling was used for this set of Bloch simulations (transverse magnetization was set to 0 at the end of each TR<sub>1,2</sub>).
</b>
</p>
</div>

In [ ]:
%% MATLAB/OCTAVE CODE
% Code used to generate the plot

clear all

% Load precomputed Bloch simulations
params = {'F', 'T1'};
MTRs = load('../blog/res/outMTR.mat');
MTRs = MTRs.outMTR;

%{
%% Get parameters that give the same MTR
% Simulate complete qMT-SPGR protocol
Model_qmtSPGR = qmt_spgr;
Model_qmtSPGR.Prot.MTdata.Mat = [490, 1200]; % 490°, 1.2 kHz

% Timing - TR = 32 ms
Model_qmtSPGR.Prot.TimingTable.Mat(1) = 0.0120;
Model_qmtSPGR.Prot.TimingTable.Mat(2) = 0.0032;
Model_qmtSPGR.Prot.TimingTable.Mat(3) = 0.0018;
Model_qmtSPGR.Prot.TimingTable.Mat(4) = 0.0150;
Model_qmtSPGR.Prot.TimingTable.Mat(5) = 0.0320;

Model_qmtSPGR.options.Readpulsealpha = 6;

% Set simulation options
Opt.Method = 'Analytical equation';
Opt.ResetMz = false;
Opt.SNR = 1000;

% Input parameters
% MTon
x = struct;
x.F = 0.16;
x.kr = 30;
x.R1f = 1;
x.R1r = 1;
x.T2f = 0.03;
x.T2r = 1.3e-05;

% Timing parameters for vfa_t1.analytical_solution function
% PDw - MToff
PDw_Model = vfa_t1;
paramsPDw.EXC_FA = 6;
paramsPDw.TR = 32; % ms

%%%%%%% VARY F AND R1F %%%%%%%
F = 0.05:0.02:0.40;
T1 = 0.5:0.05:1.9;
R1f = 1./T1;

% Get MTR
for ii=1:length(F)
    for jj=1:length(R1f)
        % Signal
        x.F = F(ii);
        x.R1f = R1f(jj);
        Signal_qmtSPGR = equation(Model_qmtSPGR, x, Opt);
        % PDw - MToff
        paramsPDw.T1 = 1/x.R1f*1000; % ms
        PDw = vfa_t1.analytical_solution(paramsPDw);
        % MTon
        MT = Signal_qmtSPGR*PDw;
        
        % MTR calculation
        MTR(ii,jj) = 100*(PDw - MT)/PDw;
        
        % Matrix of parameters
        paramMatrix(ii,jj) = struct('F', x.F, 'T1', 1/x.R1f*1000);
    end
end

% Round MTR matrix
rMTR = round(MTR);

% Get indices with equal MTR values
cnt = 1;
for i=1:100
    idx{i} = find(rMTR==i);
    if ~isempty(idx{i})
        repeatedMTR = length(idx{i});
        for j=1:repeatedMTR
            outMTR(cnt,1) = rMTR(idx{i}(j));
            outMTR(cnt,2) = paramMatrix(idx{i}(j)).F;
            outMTR(cnt,3) = paramMatrix(idx{i}(j)).T1;
            cnt = cnt + 1;
        end
    end
end
%}

In [ ]:
%get MTRs --from Octave
%get params --from Octave

In [ ]:
# PYTHON CODE

init_notebook_mode(connected=True)

data_F = [dict(
        visible = True,
        mode = 'markers',
        marker=dict(color='royalblue'),
        x = abs(np.squeeze(np.asarray(MTRs[:,0]))),
        y = abs(np.squeeze(np.asarray(MTRs[:,1]))),
        name = 'F',
        text = 'F',
        hoverinfo = 'x+y+text')]

data_T1 = [dict(
        visible = False,
        mode = 'markers',
        marker=dict(color='royalblue'),
        x = abs(np.squeeze(np.asarray(MTRs[:,0]))),
        y = abs(np.squeeze(np.asarray(MTRs[:,2]))),
        name = 'T<sub>1</sub>',
        text = 'T<sub>1</sub>',
        hoverinfo = 'x+y+text')]

data = data_F + data_T1

buttons = []
for i, label in enumerate(params):
    visibility = [False] * len(data)
    for j in np.arange(0,len(data),2):
        visibility[i+j] = True
    button = dict(
                label =  label,
                method = 'update',
                args = [{'visible': visibility}])
    buttons.append(button)
        
updatemenus = list([
    dict(active=0,
         x=0.3,
         y=1.1,
         buttons=buttons
    )
])

layout = go.Layout(
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=80,
        b=60,
        t=60,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='MTR (%)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='Parameter',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        range=[0, 100],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=True,
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.5,
        y=0.9,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)
layout['updatemenus'] = updatemenus

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

<div class=blog_body>
<p style="text-align:justify;">
In practice, gradient and RF spoiling are important parameters to consider in an AFI experiment. A combination of both spoilings (Zur et al. 1991; Bernstein et al. 2004) are typically recommended, and Figure 4 shows how it resembles more to the ideal spoiling case. 
</p>
</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 4. Signal 1 (solid lines) and Signal 2 (dashed lines) curves estimated using Bloch simulations for three categories of signal spoiling: (1) ideal spoiling (blue), gradient & RF Spoiling (red), and no spoiling (orange). Simulations details: TR<sub>1</sub> = 20 ms, TR<sub>2</sub> = 100 ms, T<sub>1</sub> = 900 ms, T<sub>2</sub> = 100 ms, TE = 5 ms, 100 spins. For the ideal spoiling case, the transverse magnetization is set to zero at the end of each TR. For the gradient & RF spoiling case, each spin is rotated by different increments of phase (2𝜋 / # of spins) to simulate complete decoherence from gradient spoiling, and the RF phase of the excitation pulse is  ɸ<sub><i>n</i></sub> = ɸ<sub><i>n</i>-1</sub> + <i>n</i>ɸ<sub>0</sub> = ½ ɸ<sub>0</sub>(<i>n</i><sup>2</sup> + <i>n</i> + 2) (Bernstein et al. 2004) with ɸ<sub>0</sub> = 39° (Zur et al. 1991) after each TR.
</b>
</p>
</div>

In [ ]:
%% MATLAB/OCTAVE CODE
% Code used to generate the data required for Figure 3 of the blog post

clear all

%%%%%%% NUMBER OF REPETITIONS: BLOCH SIMULATIONS %%%%%%%
numPulses = [1:1:10, 20:10:90, 100:100:500];
%%%%%%% DIFFERENT F VALUES: NORMAL WM, NAWM, AND LESION %%%%%%%
F = [0.215,  0.199, 0.112, 0.090];

% Load precomputed Bloch simulations
signalMTsat= load('../blog/res/MTsatBloch.mat');
MTsatBloch = signalMTsat.MTsatBloch;
signalMTR= load('../blog/res/MTRBloch.mat');
MTRBloch = signalMTR.MTRBloch;

%{
%% Get MTR, and MTsat using Bloch simulations
% Simulate complete qMT-SPGR protocol
Model_qmtSPGR = qmt_spgr;
Model_qmtSPGR.Prot.MTdata.Mat = [490, 1200]; % 490�, 1.2 kHz

% Timing - TR = 32 ms
Model_qmtSPGR.Prot.TimingTable.Mat(1) = 0.0120;
Model_qmtSPGR.Prot.TimingTable.Mat(2) = 0.0032;
Model_qmtSPGR.Prot.TimingTable.Mat(3) = 0.0018;
Model_qmtSPGR.Prot.TimingTable.Mat(4) = 0.0150;
Model_qmtSPGR.Prot.TimingTable.Mat(5) = 0.0320;

Model_qmtSPGR.options.Readpulsealpha = 6;

% Set simulation options
Opt.Method = 'Block equation';
Opt.ResetMz = false;
Opt.SNR = 1000;

% Input parameters
% MTon
x = struct;
x.F = 0.16; % Vary F
x.kr = 30;
x.R1f = 1; % Vary R1f
x.R1r = 1;
x.T2f = 0.03;
x.T2r = 1.3e-05;

% Timing parameters for vfa_t1.analytical_solution function
% PDw - MToff
PDw_Model = vfa_t1;
paramsPDw.EXC_FA = 6;
paramsPDw.TR = 32; % ms

% T1w
T1w_Model = vfa_t1; 
paramsT1w.EXC_FA = 20;
paramsT1w.TR = 18; % ms

% Timing parameters for MTSAT_exec function
PDParams(1) = paramsPDw.EXC_FA;
PDParams(2) = paramsPDw.TR;
T1Params(1) = paramsT1w.EXC_FA;
T1Params(2) = paramsT1w.TR;
MTParams(1) = 6;
MTParams(2) = 32;
B1Params(1) = 1;

% PDw - MToff
paramsPDw.T1 = 1/x.R1f*1000; % ms
PDw = vfa_t1.analytical_solution(paramsPDw);

% T1w
paramsT1w.T1 = 1/x.R1f*1000; % ms
T1w = vfa_t1.analytical_solution(paramsT1w);

%%%%%%% NUMBER OF REPETITIONS: BLOCH SIMULATIONS %%%%%%%
numPulses = [1:1:10, 20:10:90, 100:100:500];
%%%%%%% DIFFERENT F VALUES: NORMAL WM, NAWM, AND LESION %%%%%%%
F = [0.215,  0.199, 0.112, 0.090];
MTR = zeros(length(F),length(numPulses));
MTsat = zeros(length(F),length(numPulses));
qMT = zeros(length(F),length(numPulses));
% Get signal using different F and T1w
for ii=1:length(numPulses)
    for jj=1:length(F)
        % Set number of pulses
        Model_qmtSPGR.options.MT_Pulse_NofMTpulses = numPulses(ii);
        
        % Signal (MTon is the only changing
        x.F = F(jj);
        Signal_qmtSPGR = equation(Model_qmtSPGR, x, Opt);
        
        % MTon
        MT = Signal_qmtSPGR*PDw;
        qMT(jj,ii) = MT;
        
        % MTR calculation
        MTR(jj,ii) = 100*(PDw - MT)/PDw;
        
        % MTsat calculation
        dataMTsat.PDw = PDw;
        dataMTsat.T1w = T1w;
        dataMTsat.MTw = MT;
        [MTsaturation,~] = MTSAT_exec(dataMTsat, MTParams, PDParams, T1Params, B1Params);
        MTsat(jj,ii) = MTsaturation;
    end
end
MTRBloch = MTR;
MTsatBloch = MTsat;
%}

In [ ]:
%get numPulses --from Octave
%get F --from Octave
%get MTsatBloch --from Octave
%get MTRBloch --from Octave

In [ ]:
# PYTHON CODE
init_notebook_mode(connected=True)

data_MTR = [dict(
    visible = False,
    line=dict(color='royalblue'),
    x = F,
    y = abs(np.squeeze(np.asarray(MTRBloch[:,ii]))),
    name = 'MTR',
    text = 'MTR',
    hoverinfo = 'x+y+text') for ii in range(len(numPulses))]

data_MTR[7]['visible'] = True

data_MTsat = [dict(
    visible = False,
    line=dict(color='firebrick'),
    x = F,
    y = abs(np.squeeze(np.asarray(MTsatBloch[:,ii]))),
    name = 'MTsat',
    text = 'MTsat',
    xaxis='x2',
    yaxis='y2',
    hoverinfo = 'x+y+text') for ii in range(len(numPulses))]

data_MTsat[7]['visible'] = True

data = data_MTR + data_MTsat

steps = []
for i in range(len(numPulses)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data_MTR)],
        label = str(numPulses[i])
        )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.02,
    active = 6,
    currentvalue = {"prefix": "# MT pulses: <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps)]

layout = go.Layout(
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=80,
        b=60,
        t=60,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='Pool size ratio, F',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='MTR (%)',
            font=dict(
                family='Times New Roman',
                size=22,
                color='royalblue'
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.5004254919715793,
            y=1.15,
            showarrow=False,
            text=False,
            visible=False,
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.1,
            y=0.5,
            showarrow=False,
            text='MTsat (a.u.)',
            font=dict(
                family='Times New Roman',
                size=22,
                color='firebrick'
            ),
            xref='paper',
            yref='paper',
            textangle=-90,
        )
    ],
    xaxis=dict(
        autorange=False,
        range=[0, F[0]+0.1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0,100],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    xaxis2=dict(
        autorange=False,
        range=[0, F[0]+0.1],
        showgrid=False,
        showticklabels=False,
        mirror=True,
        overlaying= 'x',
        anchor= 'y2',
        side= 'top',
        linecolor='black',
        linewidth=2
    ),
    yaxis2=dict(
        autorange=False,
        range=[0,10],
        showgrid=False,
        overlaying= 'y',
        anchor= 'x',
        side= 'right',
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.05,
        y=0.85,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ),
    sliders = sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

<center> <h2 style="font-family:timesnewroman;font-size:30px">Data Fitting</h2> </center>

<div class=blog_body>
<p style="text-align:justify;">
The ratio of Equations 1 and 2, gives rise to Equation 3 that depends on the parameters T<sub>1</sub>, TR<sub>1</sub>, TR<sub>2</sub> and the excitation flip angle (<i>θ</i>). 
</p>

<p style="text-align:justify;">
<center><img src="equation3.png" style="width:700px;height:80px;margin-bottom: 50px;margin-top: 50px"></center>
</p>

<p style="text-align:justify;">
Equation 3 can be simplified if the Taylor series expansion of the exponential function is used, followed by a first-order approximation to its terms. For this expansion, short TR<sub>1</sub> and TR<sub>2</sub> (TR<sub>1</sub> < T<sub>1</sub> and TR<sub>2</sub> < T<sub>1</sub>) are assumed to approximate the signal intensities ratio (Equation 4) where n = TR<sub>2</sub>/TR<sub>1</sub>.
</p>

<p style="text-align:justify;">
<center><img src="equation4.png" style="width:700px;height:50px;margin-bottom: 50px;margin-top: 50px"></center>
</p>

<p style="text-align:justify;">
Finally, a measure of the actual flip-angle (<i>θ</i>) can be achieved by solving Equation 4 to obtain Equation 5 which only depends on the signal intensities ratio (r = S<sub>2</sub>/S<sub>1</sub>) and the parameters TR<sub>1</sub> and TR<sub>2</sub>. 
</p>

<p style="text-align:justify;">
<center><img src="equation5.png" style="width:700px;height:50px;margin-bottom: 50px;margin-top: 50px"></center>
</p>

<p style="text-align:justify;">
Equation 5 is an approximation to the complete analytical equation (Equation 4). Therefore, the accuracy of the signal ratio between Equation 4 and 5 is assessed through simulations where a range of sequence parameters are used. First, the effect of relaxation time T<sub>1</sub> is simulated and the results are displayed in Figure 5.
</p>

</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 5. Effect of the relaxation time T<sub>1</sub> on the ratio r. Signal intensities ratio is plotted as a function of the flip angle for the complete analytical equation (Equation 4 - blue) and the first-order approximation (Equation 5 - orange). AFI simulation details: TR<sub>1</sub> = 20 ms, TR<sub>2</sub> = 100 ms and variable T<sub>1</sub>.
</b>
</div>

In [ ]:
%% MATLAB/OCTAVE CODE
% Code used to generate the data required for Figure 5 of the blog post

clear all

%% Setup parameters
% All times are in milliseconds
% All flip angles are in degrees

%% Calculate signals
%
% To see all the options available, run `help b1_afi.analytical_solution`

T1_range = [10:10:200, 200:400:2000];

params.TR1 = 20;
params.TR2 = 100;
params.EXC_FA = 1:90;
n = params.TR2/params.TR1;

for jj=1:length(T1_range)
    params.T1 = T1_range(jj);
    
    % Range of T1
    [Mz1, Mz2] = b1_afi.analytical_solution(params);
    signal1_analytical = Mz1;
    signal2_analytical = Mz2;

    r_analytical(jj,:) = signal2_analytical./signal1_analytical;
    r_approximation(jj,:) = (1 + n.*cosd(params.EXC_FA))./(n + cosd(params.EXC_FA));
end


In [ ]:
%get params --from Octave
%get T1_range --from Octave
%get r_analytical --from Octave
%get r_approximation --from Octave

In [ ]:
# PYTHON CODE

init_notebook_mode(connected=True)

data1 = [dict(
        visible = False,
        mode = 'lines',
        x = params["EXC_FA"],
        y = abs(np.squeeze(np.asarray(r_analytical[ii]))),
        name = 'Analytical',
        text = 'Analytical',
        hoverinfo = 'x+y+text') for ii in range(len(T1_range))]

data1[10]['visible'] = True

data2 = [dict(
        visible = False,
        mode = 'lines',
        x = params["EXC_FA"],
        y = abs(np.squeeze(np.asarray(r_approximation[ii]))),
        name = 'Approximation',
        text = 'Approximation',
        hoverinfo = 'x+y+text') for ii in range(len(T1_range))]

data2[10]['visible'] = True

data = data1 + data2

steps = []
for i in range(len(T1_range)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data1)],
        label = str(T1_range[i])
        )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.02,
    active = 10,
    currentvalue = {"prefix": "T1 value (ms): <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps)]

layout = go.Layout(
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=40,
        b=60,
        t=10,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='Excitation Flip Angle (°)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='Signal',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        range=[0, params['EXC_FA'][-1]],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=True,
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.5,
        y=0.9,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)


<div class=blog_body>

<p style="text-align:justify;">
The signal ratio <i>r</i> is highly insensitive to the relaxation time T<sub>1</sub>, except for the ratio computed at low T<sub>1</sub> values and large flip angles (>70°). This shows that the Taylor expansion is a good approximation to the signal ratio <i>r</i> because it is possible to get rid of the inverse quadratic T<sub>1</sub> dependance by taking the first-order terms of the expansion.
</p>

<p style="text-align:justify;">
The effect of the TR<sub>1</sub> parameter on the signal ratio is shown in Figure 6. To assess the influence of the repetition time, we vary the parameter TR<sub>1</sub> having fixed the quantity n = 5 and following the relation n = TR<sub>2</sub>/TR<sub>1</sub>. As TR<sub>1</sub> increases (> 50 ms), the approximated ratio <i>r</i> slightly deviates from the analytical approach. Although the deviation is slight only at high flip angles, a good signal ratio approximation can be achieved for a wide range of flip angles and repetition times.
</p>

</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 6. Effect of the repetition time TR<sub>1</sub> on the ratio r. Signal intensities ratio is plotted as a function of the flip angle for the complete analytical equation (Equation 4 - blue) and the first-order approximation (Equation 5 - orange). AFI simulation details: Variable TR<sub>1</sub> ranging from 10 to 60 ms, fixed ratio n = 5 and T<sub>1</sub> = 900 ms.
</b>
</div>

In [ ]:
%% MATLAB/OCTAVE CODE
% Code used to generate the data required for Figure 6 of the blog post

clear all

%% Setup parameters
% All times are in milliseconds
% All flip angles are in degrees

%% Calculate signals
%
% To see all the options available, run `help b1_afi.analytical_solution`

TR1_range = 10:5:60;

params.T1 = 900;
params.EXC_FA = 1:90;
n = 5;

for jj=1:length(TR1_range)
    params.TR1 = TR1_range(jj);
    params.TR2 = n*params.TR1;
    
    % Fixed: T1 = 900 ms, n = 5
    [Mz1, Mz2] = b1_afi.analytical_solution(params);
    signal1_analytical = Mz1;
    signal2_analytical = Mz2;

    r_analytical(jj,:) = signal2_analytical./signal1_analytical;
    r_approximation(jj,:) = (1 + n.*cosd(params.EXC_FA))./(n + cosd(params.EXC_FA));
end

In [ ]:
%get params --from Octave
%get TR1_range --from Octave
%get r_analytical --from Octave
%get r_approximation --from Octave

In [ ]:
# PYTHON CODE

init_notebook_mode(connected=True)

data1 = [dict(
        visible = False,
        mode = 'lines',
        x = params["EXC_FA"],
        y = abs(np.squeeze(np.asarray(r_analytical[ii]))),
        name = 'Analytical',
        text = 'Analytical',
        hoverinfo = 'x+y+text') for ii in range(len(TR1_range))]

data1[2]['visible'] = True

data2 = [dict(
        visible = False,
        mode = 'lines',
        x = params["EXC_FA"],
        y = abs(np.squeeze(np.asarray(r_approximation[ii]))),
        name = 'Approximation',
        text = 'Approximation',
        hoverinfo = 'x+y+text') for ii in range(len(TR1_range))]

data2[2]['visible'] = True

data = data1 + data2

steps = []
for i in range(len(TR1_range)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data1)],
        label = str(TR1_range[i])
        )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.02,
    active = 2,
    currentvalue = {"prefix": "TR1 value (ms): <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps)]

layout = go.Layout(
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=40,
        b=60,
        t=10,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='Excitation Flip Angle (°)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='Signal',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        range=[0, params['EXC_FA'][-1]],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=True,
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.5,
        y=0.9,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)


<div class=blog_body>

<p style="text-align:justify;">
Finally, the effect of the parameter <i>n</i> on the signal ratio <i>r</i> (Figure 7) does not seem to significantly affect the signal ratio between the approximated equation and the analytical approach. Instead, the parameter <i>n</i> has a major impact on the sensitivity of the AFI method to variations of the flip angle. Figure 7 shows that the increase of the parameter <i>n</i> (= TR<sub>2</sub>/TR<sub>1</sub>) allows the improvement of the dynamic range of flip angles measurements. These simulations have shown that an optimal implementation of the AFI method requires a careful selection of sequence parameters.
</p>

</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 7. Effect of <i>n</i> (TR<sub>2</sub> to TR<sub>1</sub> ratio) on the ratio <i>r</i>. The signal intensities ratio is plotted as a function of the flip angle for the complete analytical equation (Equation 4 - blue) and the first-order approximation (Equation 5 - orange). AFI simulation details: Variable <i>n</i> ranging from 2 to 6, fixed TR<sub>1</sub> = 20 ms and T<sub>1</sub> = 900 ms.
</b>
</div>

In [ ]:
%% MATLAB/OCTAVE CODE
% Code used to generate the data required for Figure 7 of the blog post

clear all

%% Setup parameters
% All times are in milliseconds
% All flip angles are in degrees

%% Calculate signals
%
% To see all the options available, run `help b1_afi.analytical_solution`

n_range = 2:1:6;

params.T1 = 900;
params.TR1 = 20;
params.EXC_FA = 1:90;

for jj=1:length(n_range)
    n = n_range(jj);
    params.TR2 = n*params.TR1;
    
    % Fixed: T1 = 900 ms, TR1 = 20
    [Mz1, Mz2] = b1_afi.analytical_solution(params);
    signal1_analytical = Mz1;
    signal2_analytical = Mz2;

    r_analytical(jj,:) = signal2_analytical./signal1_analytical;
    r_approximation(jj,:) = (1 + n.*cosd(params.EXC_FA))./(n + cosd(params.EXC_FA));
end

In [ ]:
%get params --from Octave
%get n_range --from Octave
%get r_analytical --from Octave
%get r_approximation --from Octave

In [ ]:
# PYTHON CODE

init_notebook_mode(connected=True)

data1 = [dict(
        visible = False,
        mode = 'lines',
        x = params["EXC_FA"],
        y = abs(np.squeeze(np.asarray(r_analytical[ii]))),
        name = 'Analytical',
        text = 'Analytical',
        hoverinfo = 'x+y+text') for ii in range(len(n_range))]

data1[2]['visible'] = True

data2 = [dict(
        visible = False,
        mode = 'lines',
        x = params["EXC_FA"],
        y = abs(np.squeeze(np.asarray(r_approximation[ii]))),
        name = 'Approximation',
        text = 'Approximation',
        hoverinfo = 'x+y+text') for ii in range(len(n_range))]

data2[2]['visible'] = True

data = data1 + data2

steps = []
for i in range(len(n_range)):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data1)],
        label = str(n_range[i])
        )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    x = 0,
    y = -0.02,
    active = 2,
    currentvalue = {"prefix": "n value: <b>"},
    pad = {"t": 50, "b": 10},
    steps = steps)]

layout = go.Layout(
    width=580,
    height=450,
    margin=go.layout.Margin(
        l=80,
        r=40,
        b=60,
        t=10,
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='Excitation Flip Angle (°)',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='Signal',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(
        autorange=False,
        range=[0, params['EXC_FA'][-1]],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        autorange=False,
        range=[0, 1],
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.5,
        y=0.9,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    ), 
    sliders=sliders
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)


<div class=blog_body>
<p style="text-align:justify;">
Figure 8 displays an example AFI dataset and its corresponding field B<sub>1</sub> map in a healthy human brain. Although not clearly visible, both AFI images present a small Gibbs ringing artifact that is propagated and amplified due to the AFI calculation consisting of the division of both images (Boudreau et al. 2017). The ringing artifact is clearly seen in the unfiltered/raw B<sub>1</sub> field map shown in Figure 8 (right).
</p>
</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>
Figure 8. Example actual flip-angle imaging dataset (left) and a resulting raw B<sub>1</sub> map of a healthy adult brain (right). The relevant VFA protocol parameters used were: TR<sub>1</sub> = 20 ms, TR<sub>2</sub> = 100 ms and <i>θ<sub>nominal</sub></i> = 60°. The B<sub>1</sub> map (right) was fitted using the approximate <i>r</i> ratio (Equation 5).
</b>
</div>

In [ ]:
%% MATLAB/OCTAVE CODE
% Download Actual Flip-Angle Imaging brain MRI data for Figure 8 of the blog post

cmd = ['curl -L -o b1_afi.zip https://osf.io/csjgx/download?version=6'];
[STATUS,MESSAGE] = unix(cmd);
unzip('b1_afi.zip');


In [ ]:
%% MATLAB/OCTAVE CODE
% Code used to generate the data required for Figure 8 of the blog post

clear all

% Format qMRLab b1_afi model parameters, and load them into the Model object
Model = b1_afi; 
Model.Prot.Sequence.Mat = [60, 20, 100];

% Format data structure so that they may be fit by the model
data = struct();
data.AFIData1 = load_nii_data('b1_afi/AFIData1.nii.gz');
data.AFIData2 = load_nii_data('b1_afi/AFIData2.nii.gz');

FitResults = FitData(data,Model,0); % The '0' flag is so that no wait bar is shown.

In [ ]:
%% MATLAB/OCTAVE CODE
% Code used to re-orient the images to make pretty figures, and to assign variables with the axis lengths.

B1map_raw = imrotate(squeeze(FitResults.B1map_raw(:,:,14)),-90);

xAxis = [0:size(B1map_raw,2)-1];
yAxis = [0:size(B1map_raw,1)-1];

% Raw MRI AFI data
AFIData1 = imrotate(squeeze(data.AFIData1(:,:,14)),-90);
AFIData2 = imrotate(squeeze(data.AFIData2(:,:,14)),-90);

In [ ]:
%get B1map_raw --from Octave
%get AFIData1 --from Octave
%get AFIData2 --from Octave
%get xAxis --from Octave
%get yAxis --from Octave

In [ ]:
from plotly import tools

trace1 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=AFIData1,
                   colorscale='Greys',
                   showscale = False,
                   visible=False,
                   name = 'Signal1')
trace2 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=AFIData2,
                   colorscale='Greys',
                   showscale = False,
                   visible=True,
                   name = 'Signal2')
trace3 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=B1map_raw,
                   zmin=0.7,
                   zmax=1.3,
                   colorscale='RdBu',
                   xaxis='x2',
                   yaxis='y2',
                   visible=True,
                   name = 'B1 values')

data=[trace1, trace2, trace3]


updatemenus = list([
    dict(active=1,
         x = 0.09,
         xanchor = 'left',
         y = -0.15,
         yanchor = 'bottom',
         direction = 'up',
         font=dict(
                family='Times New Roman',
                size=16
            ),
         buttons=list([   
            dict(label = 'Signal 1',
                 method = 'update',
                 args = [{'visible': [True, False, True]},
                         ]),
            dict(label = 'Signal 2',
                 method = 'update',
                 args = [{'visible': [False, True, True]},
                           ]),
        ])
    )
])

layout = dict(
    width=560,
    height=345,
    margin = dict(
                t=40,
                r=50,
                b=10,
                l=50),
    annotations=[
        dict(
            x=0.07,
            y=1.15,
            showarrow=False,
            text='Input Data',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.60,
            y=1.15,
            showarrow=False,
            text='B<sub>1</sub> field map',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.12,
            y=1.15,
            showarrow=False,
            text='B<sub>1</sub>',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 0.58]),
    yaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    xaxis2 = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0.40, 0.98]),
    yaxis2 = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1], anchor='x2'),
    showlegend = False,
    autosize = False,
    updatemenus=updatemenus
)


fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-heatmap', config = config)

<div class=blog_body>
<p style="text-align:justify;">
The ringing artifact shown in Figure 8 can be attenuated by implementing a smoothing process. Figure 9 shows the raw (left) and the filtered (right) B<sub>1</sub> map where a median filter was used to smooth the field map.
</p>
</div>

<div class=figure_caption>
<p style="text-align:justify;">
<b>  
Figure 9. Raw (left) and filtered (right) B<sub>1</sub> map. A median filter of size 3x3x3 pixels was used to attenuate the Gibbs ringing artifact. 
</b>
</div>

In [ ]:
B1map_filtered = load_nii_data('../afi_blog/fitResults/FitResults_median_filter/B1map_filtered.nii.gz');
B1map_filtered = imrotate(squeeze(FitResults.B1map_filtered(:,:,14)),-90);

In [ ]:
%get B1map_filtered --from Octave

In [ ]:
from plotly import tools

trace1 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=B1map_raw,
                   zmin=0.7,
                   zmax=1.3,
                   colorscale='RdBu',
                   showscale = False,
                   visible=True,
                   name = 'B1 values')
trace2 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=B1map_filtered,
                   zmin=0.7,
                   zmax=1.3,
                   colorscale='RdBu',
                   xaxis='x2',
                   yaxis='y2',
                   visible=True,
                   name = 'B1 values')

data=[trace1, trace2]

layout = dict(
    width=560,
    height=310,
    margin = dict(
                t=40,
                r=50,
                b=10,
                l=50),
    annotations=[
        dict(
            x=0.04,
            y=1.15,
            showarrow=False,
            text='Raw B<sub>1</sub> map',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.60,
            y=1.15,
            showarrow=False,
            text='Filtered B<sub>1</sub> map',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.12,
            y=1.15,
            showarrow=False,
            text='B<sub>1</sub>',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 0.58]),
    yaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    xaxis2 = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0.40, 0.98]),
    yaxis2 = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1], anchor='x2'),
    showlegend = False,
    autosize = False,
)


fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-heatmap', config = config)

<center> <h2 style="font-family:timesnewroman;font-size:30px">Benefits and Pitfalls</h2> </center>

<div class=blog_body>
<p style="text-align:justify;">
B<sub>1</sub> mapping is of interest for diverse MRI applications and several mapping techniques have been developed. The DAM method consists of acquiring two scans at two different flip angles. To avoid the dependence of the signal on T<sub>1</sub>, long repetition times are required to allow the recovery of the longitudinal magnetization between pulses (Yarnykh 2007; Insko & Bolinger 1993). The AFI method overcomes this practical limitation by repeating the pulse sequence at a fast rate to achieve a pulsed state of magnetization and shorter time delays between pulses. In addition, due to scan-time constraints, B<sub>1</sub> mapping methods are often implemented in 2D (Chavez & Stanisz 2011). However, the accuracy of the measurements of 2D B<sub>1</sub> mapping techniques is compromised by the slice profile effects due to the problem of nonuniform excitation across slices (Yarnykh 2007; Chavez & Stanisz 2011). The AFI method on the other hand, offers a solution to this issue using a fast 3D implementation leading to scans with an excellent anatomical coverage in clinically feasible times, with an increase in signal-to-noise ratio compared to 2D multislice acquisitions.
</p>

<p style="text-align:justify;">
The performance of the AFI method is based on the following assumptions. First, the two images acquired at different times should be registered to avoid motion effects. It is also assumed that the signal is insensitive to the main magnetic field non-uniformities and chemical shift effects that are canceled out when taking the signal ratio <i>r</i> (Yarnykh 2007). Despite some clear advantages over other B<sub>1</sub> mapping techniques, the application of spoiler gradients to mitigate the T<sub>1</sub> dependence can be a limitation due to significant levels of RF power depositions (Sacolick et al. 2010). Furthermore, it is necessary to design the pulse sequence of the AFI method in the scanners, and programming experience is required to implement this technique into clinical routines.
</p>
</div>

<center> <h2 style="font-family:timesnewroman;font-size:30px">Works Cited</h2> </center>

<div class=biblio_body>
<p style="text-align:justify;">
Bernstein, M., King, K. & Zhou, X., 2004. <i>Handbook of MRI Pulse Sequences</i>, Elsevier.
</p>
    
<p style="text-align:justify;">
Boudreau, M., Tardif, C.L., Stikov, N., Sled, J.G., Lee, W. & Pike, G.B., 2017. B1 mapping for bias-correction in quantitative T1 imaging of the brain at 3T using standard pulse sequences. <i>J. Magn. Reson. Imaging</i>, 46, pp.1673-1682.
</p>
    
<p style="text-align:justify;">
Boudreau, M., Stikov, N. & Pike, G.B, 2018. B1-sensitivity analysis of quantitative magnetization transfer imaging. <i>Magn. Reson. Med.</i>, 79, pp.276-285.
</p>
    
<p style="text-align:justify;">
Chavez, S. & Stanisz, G.J., 2012. A novel method for simultaneous 3D B1 and T1 mapping: the method of slopes (MoS). <i>NMR Biomed.</i>, 25, pp.1043-1055.
</p>
    
<p style="text-align:justify;">
Deoni, S.C., 2007. High-resolution T1 mapping of the brain at 3T with driven equilibrium single pulse observation of T1 with high-speed incorporation of RF field inhomogeneities (DESPOT1-HIFI). <i>J. Magn. Reson. Imaging</i>, 26, pp.1106-1111.
</p>
    
<p style="text-align:justify;">
Ibrahim, T.S., Lee, R., Baertlein, B.A. & Robitaille, P.M., 2001. B1 field homogeneity and SAR calculations for the birdcage coil. <i>Phys Med Biol.</i>, 46(2), pp.609-619.
</p>

<p style="text-align:justify;">
Insko, E. & Bolinger, L., 1993. Mapping of the radio frequency field. <i>J. Magn. Reson. Ser. A</i>, 103(1), pp.82–85.
</p>
    
<p style="text-align:justify;">
Katscher, U., Voigt, T., Findeklee, C., Vernickel, P., Nehrke, K. & Dössel, O., 2009. Determination of electric conductivity and local SAR via B1 mapping. <i>IEEE Trans Med Imaging.</i>, 28(9), pp.1365-1374.
</p>
    
<p style="text-align:justify;">
Ropele, S., Filippi, M., Valsasina, P., Korteweg, T., Barkhof, F., Tofts, P.S., Samson, R., Miller, D.H. & Fazekas, F., 2005. Assessment and correction of B1-induced errors in magnetization transfer ratio measurements. <i>Magn. Reson. Med.</i>, 53, pp.134-140. 
</p>
    
<p style="text-align:justify;">
Sacolick, L.I., Wiesinger, F., Hancu, I. & Vogel, M.W., 2010. B1 mapping by Bloch-Siegert shift. <i>Magn. Reson. Med.</i>, 63, pp.1315-1322.
</p>
    
<p style="text-align:justify;">
Sled, J.G. & Pike, G.B., 1998. Standing-wave and RF penetration artifacts caused by elliptic geometry: an electrodynamic analysis of MRI. <i>IEEE Trans. Med. Imaging.</i>, 17(4), pp.653-662.
</p>

<p style="text-align:justify;">
Sled, J.G. & Pike, G.B., 2000. Correction for B1 and B0 variations in quantitative T2 measurements using MRI. <i>Magn. Reson. Med.</i>, 43(4), pp.589–593.
</p>

<p style="text-align:justify;">
Stollberger, R. & Wach, P., 1996. Imaging of the active B1 field in vivo. <i>Magn. Reson. Med.</i>, 35(2), pp.246–251.
</p>

<p style="text-align:justify;">
van den Bergen, B., Van den Berg, C.A., Bartels, L.W. & Lagendijk, J.J., 2007. 7 T body MRI: B1 shimming with simultaneous SAR reduction. <i>Phys Med Biol.</i>, 52(17), pp.5429-5441.
</p>

<p style="text-align:justify;">
Yarnykh, V.L. & Yuan, C. Actual flip angle imaging in the pulsed steady state. In:  <i>Proceedings of the 12th Annual Meeting of ISMRM</i>, Kyoto, Japan, 2004 (Abstract 194).
</p>

<p style="text-align:justify;">
Yarnykh, V.L., 2007. Actual Flip-Angle Imaging in the pulsed steady state: A method for rapid three-dimensional mapping of the transmitted radiofrequency field. <i>Magn. Reson. Med.</i>, 57(1), pp.192-200.
</p>
    
<p style="text-align:justify;">
Yarnykh, V.L., 2010. Optimal radiofrequency and gradient spoiling for improved accuracy of T<sub>1</sub> and B<sub>1</sub> measurements using fast steady-state techniques. <i>Magn. Reson. Med.</i>, 63(6), pp.1610–1626.
</p>
    
<p style="text-align:justify;">
Zur, Y., Wood, M.L. & Neuringer, L.J., 1991. Spoiling of transverse magnetization in steady-state sequences. <i>Magn. Reson. Med.</i>, 21(2), pp.251–263.
</p>

</div>

In [ ]:
# PYTHON CODE

display(HTML(
    '<style type="text/css">'
    '.output_subarea {'
        'display: block;'
        'margin-left: auto;'
        'margin-right: auto;'
    '}'
    '.blog_body {'
        'line-height: 2;'
        'font-family: timesnewroman;'
        'font-size: 18px;'
        'margin-left: 0px;'
        'margin-right: 0px;'
    '}'
    '.biblio_body {'
        'line-height: 1.5;'
        'font-family: timesnewroman;'
        'font-size: 18px;'
        'margin-left: 0px;'
        'margin-right: 0px;'
    '}'
    '.note_body {'
        'line-height: 1.25;'
        'font-family: timesnewroman;'
        'font-size: 18px;'
        'margin-left: 0px;'
        'margin-right: 0px;'
        'color: #696969'
    '}'
    '.figure_caption {'
        'line-height: 1.5;'
        'font-family: timesnewroman;'
        'font-size: 16px;'
        'margin-left: 0px;'
        'margin-right: 0px'
    '</style>'
))